# Vorlesung: Kopierbeziehungen und machine learning: Teil 1 Problemstellung


In [1]:
%matplotlib notebook

In [2]:
import pandas as pd
import numpy as np
from random import random,randrange
import matplotlib.pyplot as plt 
import itertools
from IPython.display import HTML
import pygraphviz as pgv

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [3]:
def show2images(pnglist,width):
    res = ''.join(['<iframe src={} width={}% height=400></iframe>'.format(png,width) for png in pnglist])
    return HTML(res)

In [4]:
def crossProbabilities(model,source,node1,node2):
    index1 = [key for key in index2kanten.keys() if (index2kanten[key][0] in [node1,node2]) & (index2kanten[key][1] in [node1,node2])]
    index2 = [key for key in dictIndex.keys() if (dictIndex[key] in index1)]
    comp_proba = model.predict_proba(source)
    res = [comp_proba[x] for x in index2]
    return res

In [5]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Konstruktion Kopierbaum

## Initialisierung

In [6]:
NMerkmale=20
NTexte=15

Die Wurzel beschreibt die Merkmale der grundlegenden Kopiervorlage.

In [7]:
def MWurzel(N):
    return([randrange(2) for i in range(N)])
NM=MWurzel(NMerkmale)
NM

[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]

In [8]:
NM2=MWurzel(NMerkmale)
NM2

[1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0]

## Modell der Merkmalskopie

Eingabe ist ein Vektor mit Merkmalen. Angegeben wird die Wahrscheinlichkeit, wie ein Merkmal mit binären Werten kopiert wird.

 Angenommen wird, dass die Wahrscheinlichkeit für die Umwandlung einer null in einer 1 p ist, während die Verwandlung von einem Merkmal mit eins in einen null mit der Häufigkeit q auftritt.
 
 http://stackoverflow.com/questions/10803135/weighted-choice-short-and-simple

In [9]:
# mit den wahrscheinlichkeiten p und q werden die Merkmale 0,1 verändert
def Binaerkopie(m,p,q):
    rd=random()
    if m==0:
        if rd<p:
            w=1
        else:
            w=0
    else:
        if rd<q:
            w=0
        else:
            w=1
    return(w)

In [10]:
def ModelKopie(M,p,q):
    copy=[Binaerkopie(M[i],p,q) for i in range(len(M))]
    return(copy)

In [11]:
# Kopie aus dem Wurzelvektor, mit einer Wahrscheinlichkeit der Merkmalsveränderung von p=0.015 für das Merkmal 0 und prob=0.3 für 1 (Verhältnis: 1/95)
p=0.015
q=0.2
ModelKopie(NM,p,q)

[1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]

# Generierung von Kopierbäumen, mit Aufzeichnung der Kopierpaare

Baut einen direkted graph auf mittels NetworkX Bibliothek.

In [12]:
def Make_Kopiergraph(IniVektor,NTexte):
# dictionary mit allen objekten ID:Merkmalsvektor
    d={0:IniVektor}
    G=pgv.AGraph(compound=True,directed=True)  # Ein Kopierbaum wird als gerichteter Graph dargestellt
    for i in range(1,NTexte):
        # Zufallsauswahl eines Textes
        pos=randrange(len(d))
        Tsel=d[pos]
        # Kopie des ausgewählen Textes, d[i] ist Merkmalsvektor des i-ten Objekts
        d[i]=ModelKopie(Tsel,p,q)
        # Kopierpaare werden in einen Graph eingetragen
        G.add_edge(pos,i)
    return(G,d)

In [13]:
G,d=Make_Kopiergraph(NM,10)
d

{0: [1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
 1: [1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
 2: [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 3: [0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
 4: [1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 5: [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0],
 6: [1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
 7: [0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 8: [1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 9: [1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]}

Um die trainierten Klassifizierer zu testen, generieren wir einen neuen Kopierbaum aus einer anderen Wurzel.

In [14]:
d.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [15]:
# die gerichteten Kanten sind
Kanten=G.edges()
Kanten

[('0', '1'),
 ('0', '3'),
 ('0', '6'),
 ('0', '8'),
 ('0', '9'),
 ('1', '2'),
 ('1', '4'),
 ('1', '5'),
 ('3', '7')]

In [16]:
G.draw('g.png',prog='dot')
show2images(['g.png'],50)

In [17]:
G_pruef,d_pruef=Make_Kopiergraph(NM2,10)
d_pruef

{0: [1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0],
 1: [1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0],
 2: [0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0],
 3: [1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0],
 4: [0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0],
 5: [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0],
 6: [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0],
 7: [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0],
 8: [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
 9: [1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

In [18]:
Kanten_pruef=G_pruef.edges()

In [19]:
G_pruef.draw('g_test.png',prog='dot')

In [20]:
show2images(['g.png','g_test.png'],50)

# Klassifikationsaufgabe

Klassifikationsaufgaben besteht darin, aus den möglichen Kombinationen aller Knoten die gerichteten Kanten zu identifizieren. Die Lernmenge für  die Klassifikation wird deshalb darin bestehen, aus der Menge der Kombinationen solche Label mit einer direkten Verbindung mit einer eins zu kennzeichnen, während alle anderen Kombinationen eine null erhalten.

Die Merkmale für die Klassifikation besteht aus der Differenz der Merkmalvektor beider gerichteter Paare. Es ergeben sich insgesamt vier Werte, die für jedes Merkmal berechnet werden müssen.

In der Liste Kanten sind alle Kanten. Die Merkmalliste der Objekte haben alle die gleiche Länge. Die Menge alle betrachteten möglichen Kopierpaare ist die Kombination der Objekte mit allen anderen ausser sich selbst. Bei n Objekten ergeben sich somit n*(n-1) Paare.

In [21]:
def diffv(d,i,j): # vergleich zweier Merkmalsvektoren i und j
    d1=d[i] # vektor des ersten objekts
    d2=d[j] # vektor des zweiten objekts
    diffv1=[]
    for u in range(len(d1)): # vergleich der Vektoren
        if d1[u]==0 and d2[u]==0:
            dv=0
        if d1[u]==1 and d2[u]==0:
            dv=1
        if d1[u]==0 and d2[u]==1:
            dv=2
        if d1[u]==1 and d2[u]==1:
            dv=3
        diffv1.append(dv)
    return(diffv1)

def categorie(i,j,Kanten):
    if (str(i),str(j)) in Kanten:
        res = 1
    elif (str(j),str(i)) in Kanten:
        res = 2
    else:
        res = 0
    return res

def Vergleich(d,i,j,Kanten): # vergleich zweier Objekte
    dd=(i,j,diffv(d,i,j),categorie(i,j,Kanten))
    return(dd)

In [22]:
# Vergleich der ersten beiden Objekte
Vergleich(d,0,1,Kanten)

(0, 1, [3, 3, 0, 0, 0, 3, 3, 3, 1, 0, 0, 0, 3, 3, 0, 0, 3, 0, 0, 0], 1)

In [23]:
Vergleich(d,0,2,Kanten)

(0, 2, [1, 3, 0, 0, 0, 3, 3, 1, 1, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0], 0)

In [48]:
Vergleich(d,0,3,Kanten)

(0, 3, [1, 3, 0, 0, 0, 3, 1, 3, 3, 0, 0, 0, 3, 3, 0, 0, 3, 0, 0, 0], 1)

In [49]:
Vergleich(d,1,2,Kanten)

(1, 2, [1, 3, 0, 0, 0, 3, 3, 1, 0, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0], 1)

In [50]:
l=[]
for j in range(len(d)):
    for k in range(len(d)):
        if k!=j:
            l.append(Vergleich(d,j,k,Kanten))

In [51]:
l

[(0, 1, [3, 3, 0, 0, 0, 3, 3, 3, 1, 0, 0, 0, 3, 3, 0, 0, 3, 0, 0, 0], 1),
 (0, 2, [1, 3, 0, 0, 0, 3, 3, 1, 1, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0], 0),
 (0, 3, [1, 3, 0, 0, 0, 3, 1, 3, 3, 0, 0, 0, 3, 3, 0, 0, 3, 0, 0, 0], 1),
 (0, 4, [3, 3, 0, 0, 0, 3, 3, 3, 1, 0, 0, 0, 1, 3, 0, 0, 3, 0, 0, 0], 0),
 (0, 5, [3, 3, 0, 0, 0, 3, 3, 1, 1, 0, 0, 0, 3, 3, 0, 0, 3, 0, 0, 0], 0),
 (0, 6, [3, 3, 0, 0, 0, 1, 3, 3, 1, 0, 0, 0, 3, 1, 0, 2, 3, 0, 0, 0], 1),
 (0, 7, [1, 3, 0, 0, 0, 3, 1, 3, 3, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0], 0),
 (0, 8, [3, 3, 0, 0, 0, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0], 1),
 (0, 9, [3, 3, 0, 0, 0, 3, 3, 1, 3, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0], 1),
 (1, 0, [3, 3, 0, 0, 0, 3, 3, 3, 2, 0, 0, 0, 3, 3, 0, 0, 3, 0, 0, 0], 2),
 (1, 2, [1, 3, 0, 0, 0, 3, 3, 1, 0, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0], 1),
 (1, 3, [1, 3, 0, 0, 0, 3, 1, 3, 2, 0, 0, 0, 3, 3, 0, 0, 3, 0, 0, 0], 0),
 (1, 4, [3, 3, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 1, 3, 0, 0, 3, 0, 0, 0], 1),
 (1, 5, [3, 3, 0, 0, 0, 3, 3, 1, 0, 0,

In [26]:
from collections import Counter

countL = [dict(Counter(x[2])) for x in l]
countL[:10]

[{0: 11, 1: 1, 3: 8},
 {0: 11, 1: 4, 3: 5},
 {0: 11, 1: 2, 3: 7},
 {0: 11, 1: 2, 3: 7},
 {0: 11, 1: 2, 3: 7},
 {0: 10, 1: 3, 2: 1, 3: 6},
 {0: 11, 1: 3, 3: 6},
 {0: 11, 1: 1, 3: 8},
 {0: 11, 1: 2, 3: 7},
 {0: 11, 2: 1, 3: 8}]

# ML

* Aufteilung der Simulationsklasse
* Lernen
* Vorhersagen
* Bewerten
* Crossvalidation

## Trainings-Datensatz erstellen


In [27]:
df = pd.DataFrame(l)
df = df.rename(columns={0:'dia1',1:'dia2',2:'diff', 3:'kopie'})
df.head(3)

,dia1,dia2,diff,kopie
0,0,1,"[3, 3, 0, 0, 0, 3, 3, 3, 1, 0, 0, 0, 3, 3, 0, ...",1
1,0,2,"[1, 3, 0, 0, 0, 3, 3, 1, 1, 0, 0, 0, 3, 3, 0, ...",0
2,0,3,"[1, 3, 0, 0, 0, 3, 1, 3, 3, 0, 0, 0, 3, 3, 0, ...",1


In [28]:
serDiff_train = df['diff'].apply(lambda r: Counter(r))

diff = pd.DataFrame(serDiff_train.to_dict()).transpose().fillna(0)
diff.head(3)

,0,1,2,3
0,11.0,1.0,0.0,8.0
1,11.0,4.0,0.0,5.0
2,11.0,2.0,0.0,7.0


In [29]:
dfTrain = df.merge(diff,left_index=True, right_index=True, how='outer')
dfTrain = dfTrain.drop('diff',axis=1)
dfTrain.head(5)

,dia1,dia2,kopie,0,1,2,3
0,0,1,1,11.0,1.0,0.0,8.0
1,0,2,0,11.0,4.0,0.0,5.0
2,0,3,1,11.0,2.0,0.0,7.0
3,0,4,0,11.0,2.0,0.0,7.0
4,0,5,0,11.0,2.0,0.0,7.0


Zur Rekonstruktion der Baumstruktur generieren wir ein Dict aller möglicher Kanten. Später wird das mit den vorhergesagten Kanten verglichen und daraus eine Baumdarstellung generiert. 

In [30]:
dfKan= list(zip(dfTrain['dia1'],dfTrain['dia2']))
dfKan = pd.DataFrame([dfKan]).transpose().rename(columns={0:'Kanten'})
pre = dfKan.to_dict()
index2kanten = pre['Kanten']

Das Ziel des Trainings ist es, zu entscheiden, ob eine Kopierbeziehung vorliegt oder nicht. Daher trennen wir die Spalte mit der Einordnung ob eine Kopie vorliegt oder nicht, als Label-Vektor ab.

In [31]:
y_train = dfTrain['kopie']
y_train.head()

0    1
1    0
2    1
3    0
4    0
Name: kopie, dtype: int64

Da die Namen der Vektoren für die Klassifizierung nicht von Bedeutung sind, schneiden wir diese Spalten ab.

In [32]:
X_train = dfTrain.drop('kopie',axis=1).drop('dia1',axis=1).drop('dia2',axis=1)
X_train.head(3)

,0,1,2,3
0,11.0,1.0,0.0,8.0
1,11.0,4.0,0.0,5.0
2,11.0,2.0,0.0,7.0
